In [16]:
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgbm
import xgboost as xgb
from sklearn.model_selection import KFold
from imblearn.over_sampling import SVMSMOTE
import os

In [17]:
mfcc = pd.read_csv("drive/MyDrive/ML_Boost/127_features/minmax_mfcc_features.csv")
chroma = pd.read_csv("drive/MyDrive/ML_Boost/127_features/minmax_chroma_features.csv")
mel = pd.read_csv("drive/MyDrive/ML_Boost/127_features/minmax_mel_features.csv")
zcr = pd.read_csv("drive/MyDrive/ML_Boost/127_features/minmax_zcr_features.csv")

train_df = pd.concat([zcr.iloc[:, :-1],
                      mfcc.iloc[:, :-1],
                      chroma.iloc[:, :-1], mel], axis=1)

X, y = train_df.iloc[:, :-1].values, train_df.iloc[:, -1].values

print(X.shape, y.shape)

smote = SVMSMOTE(sampling_strategy=1, random_state=42)

(4068, 154) (4068,)


### lightgbm

In [20]:
params = {"objective": "binary",
              "boosting_type": "gbdt",
              "metric" : ["auc", "binary_logloss"],
              "learning_rate": 0.03,
              "subsample": 0.68,
              "tree_learner": "serial",
              "colsample_bytree": 0.28,
              "early_stopping_rounds": 100,
              "subsample_freq": 1,
              "reg_lambda": 2,
              "reg_alpha": 1,
              "num_leaves": 500,
              "random_state": 42,}

seeds = range(10)
fold_seed = 42
n_splits = 10

In [18]:
import matplotlib.pyplot as plt
root = "drive/MyDrive/ML_Boost"
# os.makedirs("drive/MyDrive/ML_Boost/127_weights/modelsLgbm")

In [21]:
for seed in seeds:

        print("---seed: ", seed)

        kfolds = KFold(n_splits=n_splits, random_state=fold_seed, shuffle=True)

        for f, (trn_idx, val_idx) in enumerate(kfolds.split(X=X, y=y)):

            kf_x_train, kf_y_train = X[trn_idx], y[trn_idx]
            kf_x_train, kf_y_train = smote.fit_resample(kf_x_train, kf_y_train)
            kf_x_valid, kf_y_valid = X[val_idx], y[val_idx]
            
            print(kf_x_train.shape, kf_y_train.shape)
            
            dtrain = lgbm.Dataset(kf_x_train, kf_y_train)
            dvalid = lgbm.Dataset(kf_x_valid, kf_y_valid)
            
            params["random_state"] = seed
            params["num_leaves"] = 500 + seed * 25

            evals_result = {}  # to record eval results for plotting

            model = lgbm.train(params, 
                              dtrain, 
                              num_boost_round=10000,
                              early_stopping_rounds=params["early_stopping_rounds"],
                              valid_sets=(dtrain, dvalid), 
                              valid_names=("train", "valid"),
                              evals_result=evals_result,
                              verbose_eval = 100)

            f = open(f"{root}/127_weights/modelsLgbm/model_{seed + 1}_{f + 1}.pkl", "wb")
            pickle.dump(model, f)
            f.close()

            # lgbm.plot_metric(model)
            print('Plot metrics AUC')
            ax = lgbm.plot_metric(evals_result, metric='auc')
            plt.show()
            print('Plot metrics logloss')
            ax = lgbm.plot_metric(evals_result, metric='binary_logloss')
            plt.show()

            print('=' * 80)

Output hidden; open in https://colab.research.google.com to view.

### XGBoost

In [22]:
# params = {
#               "boosting_type": "gbdt",
#               "eval_metric" : ["auc", "logloss"],
#               "learning_rate": 0.03,
#               "subsample": 0.68,
#               "tree_learner": "serial",
#               "colsample_bytree": 0.28,
#               "early_stopping_rounds": 3,
#               "subsample_freq": 1,
#               "reg_lambda": 2,
#               "reg_alpha": 1,
#               "num_leaves": 500,
#               "random_state": 42,}

params = {
              "objective": "binary:logistic",
              "eval_metric" : ["auc", "logloss"],
              "learning_rate": 0.08,
              "eta": 0.2,
              "gamma": 0,
              "subsample": 0.68,
              "colsample_bytree": 0.28,
              "lambda": 1,
              "lambda_bias": 1,
              "alpha": 1,
              "verbose": 2,
              "random_state": 42,
              }

seeds = range(10)
fold_seed = 42
n_splits = 10

In [23]:
from matplotlib import pyplot
os.makedirs("drive/MyDrive/ML_Boost/127_weights/modelsXGboost")

In [24]:
for seed in seeds:

        print("---seed: ", seed)

        kfolds = KFold(n_splits=n_splits, random_state=fold_seed, shuffle=True)

        for f, (trn_idx, val_idx) in enumerate(kfolds.split(X=X, y=y)):

            kf_x_train, kf_y_train = X[trn_idx], y[trn_idx]
            kf_x_train, kf_y_train = smote.fit_resample(kf_x_train, kf_y_train)
            kf_x_valid, kf_y_valid = X[val_idx], y[val_idx]
            
            print(kf_x_train.shape, kf_y_train.shape)
            
            dtrain = xgb.DMatrix(kf_x_train, kf_y_train)
            
            dvalid = xgb.DMatrix(kf_x_valid, kf_y_valid)

            watchlist = [(dvalid, 'valid'), (dtrain, 'train')]
            
            params["random_state"] = seed
            params["num_leaves"] = 500 + seed * 25

            evals_result = {}  # to record eval results for plotting

            model = xgb.train(params, 
                              dtrain, 
                              20,
                              watchlist,
                              evals_result = evals_result
                              )

            f = open(f"{root}/127_weights/modelsXGboost/model_{seed + 1}_{f + 1}.pkl", "wb")
            pickle.dump(model, f)
            f.close()

            # print('Plot metrics recorded during training...')
            # results = model.evals_result()
            # epochs = len(results['validation_0']['error'])
            # x_axis = range(0, epochs)
            # fig, ax = pyplot.subplots()
            # ax.plot(x_axis, results['validation_0']['auc'], label='Train')
            # ax.plot(x_axis, results['validation_1']['auc'], label='Test')
            # ax.legend()
            # pyplot.ylabel('AUC')
            # pyplot.title('XGBoost AUC')
            # pyplot.show()

            print('=' * 80)

---seed:  0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6126, 154) (6126,)
[0]	valid-auc:0.738347	valid-logloss:0.665339	train-auc:0.888039	train-logloss:0.655864
[1]	valid-auc:0.792065	valid-logloss:0.634439	train-auc:0.929995	train-logloss:0.617604
[2]	valid-auc:0.794706	valid-logloss:0.609069	train-auc:0.939704	train-logloss:0.586389
[3]	valid-auc:0.804808	valid-logloss:0.584494	train-auc:0.946577	train-logloss:0.555746
[4]	valid-auc:0.804556	valid-logloss:0.565393	train-auc:0.948139	train-logloss:0.528841
[5]	valid-auc:0.815267	valid-logloss:0.550787	train-auc:0.952645	train-logloss:0.510463
[6]	valid-auc:0.812982	valid-logloss:0.536006	train-auc:0.952662	train-logloss:0.49176
[7]	valid-auc:0.816755	valid-logloss:0.520732	train-auc:0.95688	train-logloss:0.47029
[8]	valid-auc:0.823084	valid-logloss:0.507003	train-auc:0.959508	train-logloss:0.45464
[9]	valid-auc:0.824384	valid-logloss:0.496282	train-auc:0.962005	train-logloss:0.437067
[10]	valid-auc:0.824153	valid-logloss:0.483896	train-auc:0.96332	train-logloss:0.421417
[11]	valid-auc:0

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6120, 154) (6120,)
[0]	valid-auc:0.818844	valid-logloss:0.660876	train-auc:0.888531	train-logloss:0.655227
[1]	valid-auc:0.864394	valid-logloss:0.626111	train-auc:0.931674	train-logloss:0.61698
[2]	valid-auc:0.883958	valid-logloss:0.597757	train-auc:0.9378	train-logloss:0.585134
[3]	valid-auc:0.881811	valid-logloss:0.574387	train-auc:0.943534	train-logloss:0.557335
[4]	valid-auc:0.882483	valid-logloss:0.554923	train-auc:0.948068	train-logloss:0.532808
[5]	valid-auc:0.882917	valid-logloss:0.53522	train-auc:0.952056	train-logloss:0.506827
[6]	valid-auc:0.882743	valid-logloss:0.517687	train-auc:0.952244	train-logloss:0.485497
[7]	valid-auc:0.886431	valid-logloss:0.504659	train-auc:0.955498	train-logloss:0.468641
[8]	valid-auc:0.889728	valid-logloss:0.488383	train-auc:0.957217	train-logloss:0.449985
[9]	valid-auc:0.890508	valid-logloss:0.473664	train-auc:0.958937	train-logloss:0.43164
[10]	valid-auc:0.892873	valid-logloss:0.458617	train-auc:0.960073	train-logloss:0.414723
[11]	valid-auc:0

...

(6100, 154) (6100,)
[0]	valid-auc:0.766871	valid-logloss:0.661819	train-auc:0.879854	train-logloss:0.654143
[1]	valid-auc:0.800464	valid-logloss:0.630944	train-auc:0.927908	train-logloss:0.617735
[2]	valid-auc:0.822275	valid-logloss:0.601077	train-auc:0.939763	train-logloss:0.582581
[3]	valid-auc:0.840455	valid-logloss:0.57259	train-auc:0.944796	train-logloss:0.552899
[4]	valid-auc:0.841073	valid-logloss:0.555571	train-auc:0.953112	train-logloss:0.527652
[5]	valid-auc:0.849669	valid-logloss:0.533047	train-auc:0.955323	train-logloss:0.502206
[6]	valid-auc:0.857079	valid-logloss:0.515301	train-auc:0.956406	train-logloss:0.480726
[7]	valid-auc:0.8678	valid-logloss:0.499447	train-auc:0.959841	train-logloss:0.460399
[8]	valid-auc:0.873283	valid-logloss:0.483855	train-auc:0.960808	train-logloss:0.44207
[9]	valid-auc:0.87148	valid-logloss:0.473797	train-auc:0.961619	train-logloss:0.426532
[10]	valid-auc:0.873432	valid-logloss:0.461165	train-auc:0.961957	train-logloss:0.411602
[11]	valid-auc:0

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6118, 154) (6118,)
[0]	valid-auc:0.784091	valid-logloss:0.661775	train-auc:0.887918	train-logloss:0.655043
[1]	valid-auc:0.817781	valid-logloss:0.629809	train-auc:0.922573	train-logloss:0.620831
[2]	valid-auc:0.833601	valid-logloss:0.603923	train-auc:0.929702	train-logloss:0.591554
[3]	valid-auc:0.860294	valid-logloss:0.576211	train-auc:0.944842	train-logloss:0.560787
[4]	valid-auc:0.873685	valid-logloss:0.55275	train-auc:0.948887	train-logloss:0.533974
[5]	valid-auc:0.87119	valid-logloss:0.532784	train-auc:0.953317	train-logloss:0.507539
[6]	valid-auc:0.873908	valid-logloss:0.514766	train-auc:0.953969	train-logloss:0.485701
[7]	valid-auc:0.883645	valid-logloss:0.496034	train-auc:0.95557	train-logloss:0.465189
[8]	valid-auc:0.885651	valid-logloss:0.481366	train-auc:0.956305	train-logloss:0.448557
[9]	valid-auc:0.884091	valid-logloss:0.469198	train-auc:0.956628	train-logloss:0.431762
[10]	valid-auc:0.884559	valid-logloss:0.458208	train-auc:0.957272	train-logloss:0.417289
[11]	valid-auc

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6094, 154) (6094,)
[0]	valid-auc:0.859086	valid-logloss:0.660593	train-auc:0.881836	train-logloss:0.656347
[1]	valid-auc:0.880682	valid-logloss:0.629824	train-auc:0.911289	train-logloss:0.626135
[2]	valid-auc:0.881866	valid-logloss:0.604009	train-auc:0.931893	train-logloss:0.594779
[3]	valid-auc:0.884628	valid-logloss:0.577848	train-auc:0.943899	train-logloss:0.563578
[4]	valid-auc:0.88831	valid-logloss:0.555187	train-auc:0.950382	train-logloss:0.535987
[5]	valid-auc:0.885154	valid-logloss:0.536222	train-auc:0.954337	train-logloss:0.512454
[6]	valid-auc:0.889178	valid-logloss:0.520607	train-auc:0.957433	train-logloss:0.493349
[7]	valid-auc:0.890283	valid-logloss:0.502768	train-auc:0.958184	train-logloss:0.472699
[8]	valid-auc:0.89486	valid-logloss:0.489274	train-auc:0.960536	train-logloss:0.456247
[9]	valid-auc:0.895518	valid-logloss:0.474694	train-auc:0.961225	train-logloss:0.43981
[10]	valid-auc:0.894834	valid-logloss:0.459347	train-auc:0.962737	train-logloss:0.422494
[11]	valid-auc

# WITH DATA 732 FEATURES

In [29]:
data_df = pd.read_csv("drive/MyDrive/ML_Boost/732_features/data.csv")
X, y = data_df.iloc[:, :-1].values, data_df.iloc[:, -1].values
smote = SVMSMOTE(sampling_strategy=1, random_state=42)

In [32]:
params = {"objective": "binary",
              "boosting_type": "gbdt",
              "metric" : ["auc", "binary_logloss"],
              "learning_rate": 0.03,
              "subsample": 0.68,
              "tree_learner": "serial",
              "colsample_bytree": 0.28,
              "early_stopping_rounds": 100,
              "subsample_freq": 1,
              "reg_lambda": 2,
              "reg_alpha": 1,
              "num_leaves": 500,
              "random_state": 42,}

seeds = range(10)
fold_seed = 42
n_splits = 10

In [30]:
# os.makedirs("drive/MyDrive/ML_Boost/732_weights/modelsLgbm")

In [33]:
for seed in seeds:

        print("---seed: ", seed)

        kfolds = KFold(n_splits=n_splits, random_state=fold_seed, shuffle=True)

        for f, (trn_idx, val_idx) in enumerate(kfolds.split(X=X, y=y)):

            kf_x_train, kf_y_train = X[trn_idx], y[trn_idx]
            kf_x_train, kf_y_train = smote.fit_resample(kf_x_train, kf_y_train)
            kf_x_valid, kf_y_valid = X[val_idx], y[val_idx]
            
            print(kf_x_train.shape, kf_y_train.shape)
            
            dtrain = lgbm.Dataset(kf_x_train, kf_y_train)
            dvalid = lgbm.Dataset(kf_x_valid, kf_y_valid)
            
            params["random_state"] = seed
            params["num_leaves"] = 500 + seed * 25

            evals_result = {}  # to record eval results for plotting

            model = lgbm.train(params, 
                              dtrain, 
                              num_boost_round=10000,
                              early_stopping_rounds=params["early_stopping_rounds"],
                              valid_sets=(dtrain, dvalid), 
                              valid_names=("train", "valid"),
                              evals_result=evals_result,
                              verbose_eval = 100)
            
            f = open(f"{root}/732_weights/modelsLgbm/model_{seed + 1}_{f + 1}.pkl", "wb")
            pickle.dump(model, f)
            f.close()

            # lgbm.plot_metric(model)
            print('Plot metrics AUC')
            ax = lgbm.plot_metric(evals_result, metric='auc')
            plt.show()
            print('Plot metrics logloss')
            ax = lgbm.plot_metric(evals_result, metric='binary_logloss')
            plt.show()

            print('=' * 80)

Output hidden; open in https://colab.research.google.com to view.

In [34]:
params = {
              "objective": "binary:logistic",
              "eval_metric" : ["auc", "logloss"],
              "learning_rate": 0.08,
              "eta": 0.2,
              "gamma": 0,
              "subsample": 0.68,
              "colsample_bytree": 0.28,
              "lambda": 1,
              "lambda_bias": 1,
              "alpha": 1,
              "verbose": 2,
              "random_state": 42,
              }

seeds = range(10)
fold_seed = 42
n_splits = 10

In [35]:
os.makedirs("drive/MyDrive/ML_Boost/732_weights/modelsXGboost")

In [36]:
for seed in seeds:

        print("---seed: ", seed)

        kfolds = KFold(n_splits=n_splits, random_state=fold_seed, shuffle=True)

        for f, (trn_idx, val_idx) in enumerate(kfolds.split(X=X, y=y)):

            kf_x_train, kf_y_train = X[trn_idx], y[trn_idx]
            kf_x_train, kf_y_train = smote.fit_resample(kf_x_train, kf_y_train)
            kf_x_valid, kf_y_valid = X[val_idx], y[val_idx]
            
            print(kf_x_train.shape, kf_y_train.shape)
            
            dtrain = xgb.DMatrix(kf_x_train, kf_y_train)
            
            dvalid = xgb.DMatrix(kf_x_valid, kf_y_valid)

            watchlist = [(dvalid, 'valid'), (dtrain, 'train')]
            
            params["random_state"] = seed
            params["num_leaves"] = 500 + seed * 25

            evals_result = {}  # to record eval results for plotting

            model = xgb.train(params, 
                              dtrain, 
                              20,
                              watchlist,
                              evals_result = evals_result
                              )

            f = open(f"{root}/732_weights/modelsXGboost/model_{seed + 1}_{f + 1}.pkl", "wb")
            pickle.dump(model, f)
            f.close()

            print('=' * 80)

---seed:  0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6126, 732) (6126,)
[0]	valid-auc:0.726798	valid-logloss:0.666212	train-auc:0.875222	train-logloss:0.658687
[1]	valid-auc:0.767228	valid-logloss:0.639795	train-auc:0.915708	train-logloss:0.628024
[2]	valid-auc:0.782005	valid-logloss:0.615674	train-auc:0.924222	train-logloss:0.602057
[3]	valid-auc:0.812877	valid-logloss:0.594346	train-auc:0.940331	train-logloss:0.573139
[4]	valid-auc:0.819647	valid-logloss:0.575403	train-auc:0.944148	train-logloss:0.550508
[5]	valid-auc:0.826731	valid-logloss:0.555402	train-auc:0.950492	train-logloss:0.527715
[6]	valid-auc:0.834004	valid-logloss:0.538624	train-auc:0.955468	train-logloss:0.507061
[7]	valid-auc:0.839579	valid-logloss:0.523528	train-auc:0.9575	train-logloss:0.488844
[8]	valid-auc:0.839453	valid-logloss:0.509736	train-auc:0.960064	train-logloss:0.470844
[9]	valid-auc:0.84507	valid-logloss:0.497296	train-auc:0.96348	train-logloss:0.453747
[10]	valid-auc:0.847963	valid-logloss:0.483821	train-auc:0.965069	train-logloss:0.438074
[11]	valid-auc:

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6120, 732) (6120,)
[0]	valid-auc:0.811773	valid-logloss:0.663895	train-auc:0.866248	train-logloss:0.660098
[1]	valid-auc:0.815678	valid-logloss:0.640286	train-auc:0.909094	train-logloss:0.628339
[2]	valid-auc:0.833529	valid-logloss:0.616974	train-auc:0.929419	train-logloss:0.60008
[3]	valid-auc:0.842877	valid-logloss:0.596064	train-auc:0.93619	train-logloss:0.574674
[4]	valid-auc:0.840968	valid-logloss:0.578799	train-auc:0.943124	train-logloss:0.551953
[5]	valid-auc:0.839602	valid-logloss:0.562879	train-auc:0.945878	train-logloss:0.530955
[6]	valid-auc:0.85318	valid-logloss:0.547053	train-auc:0.948763	train-logloss:0.513243
[7]	valid-auc:0.862376	valid-logloss:0.531226	train-auc:0.953073	train-logloss:0.494215
[8]	valid-auc:0.863786	valid-logloss:0.520269	train-auc:0.956579	train-logloss:0.476197
[9]	valid-auc:0.868471	valid-logloss:0.507607	train-auc:0.958779	train-logloss:0.459835
[10]	valid-auc:0.867343	valid-logloss:0.498003	train-auc:0.961198	train-logloss:0.443638
[11]	valid-auc

...

(6100, 732) (6100,)
[0]	valid-auc:0.718111	valid-logloss:0.665415	train-auc:0.870788	train-logloss:0.659828
[1]	valid-auc:0.802737	valid-logloss:0.64	train-auc:0.913852	train-logloss:0.628389
[2]	valid-auc:0.822522	valid-logloss:0.612699	train-auc:0.928793	train-logloss:0.598659
[3]	valid-auc:0.83633	valid-logloss:0.589647	train-auc:0.93618	train-logloss:0.572668
[4]	valid-auc:0.854165	valid-logloss:0.568327	train-auc:0.943546	train-logloss:0.549695
[5]	valid-auc:0.871357	valid-logloss:0.54927	train-auc:0.947329	train-logloss:0.528771
[6]	valid-auc:0.872246	valid-logloss:0.533916	train-auc:0.95214	train-logloss:0.508304
[7]	valid-auc:0.873703	valid-logloss:0.518436	train-auc:0.957331	train-logloss:0.48935
[8]	valid-auc:0.875951	valid-logloss:0.506046	train-auc:0.959857	train-logloss:0.472548
[9]	valid-auc:0.874667	valid-logloss:0.495021	train-auc:0.960451	train-logloss:0.457451
[10]	valid-auc:0.87684	valid-logloss:0.482512	train-auc:0.964724	train-logloss:0.441064
[11]	valid-auc:0.8750

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6118, 732) (6118,)
[0]	valid-auc:0.828365	valid-logloss:0.659216	train-auc:0.893297	train-logloss:0.655653
[1]	valid-auc:0.854234	valid-logloss:0.63187	train-auc:0.923684	train-logloss:0.623476
[2]	valid-auc:0.856506	valid-logloss:0.608233	train-auc:0.937776	train-logloss:0.596066
[3]	valid-auc:0.854679	valid-logloss:0.587536	train-auc:0.94371	train-logloss:0.570574
[4]	valid-auc:0.848685	valid-logloss:0.573186	train-auc:0.948309	train-logloss:0.547017
[5]	valid-auc:0.848017	valid-logloss:0.555719	train-auc:0.952539	train-logloss:0.525451
[6]	valid-auc:0.851627	valid-logloss:0.54118	train-auc:0.956559	train-logloss:0.505666
[7]	valid-auc:0.859514	valid-logloss:0.526277	train-auc:0.960322	train-logloss:0.487015
[8]	valid-auc:0.860071	valid-logloss:0.513993	train-auc:0.96086	train-logloss:0.47152
[9]	valid-auc:0.873373	valid-logloss:0.499413	train-auc:0.964751	train-logloss:0.452249
[10]	valid-auc:0.874866	valid-logloss:0.489499	train-auc:0.967904	train-logloss:0.437133
[11]	valid-auc:0

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6094, 732) (6094,)
[0]	valid-auc:0.807186	valid-logloss:0.667013	train-auc:0.889716	train-logloss:0.655569
[1]	valid-auc:0.811895	valid-logloss:0.642297	train-auc:0.926424	train-logloss:0.624803
[2]	valid-auc:0.849274	valid-logloss:0.617326	train-auc:0.938379	train-logloss:0.59564
[3]	valid-auc:0.863715	valid-logloss:0.598885	train-auc:0.943652	train-logloss:0.57144
[4]	valid-auc:0.873185	valid-logloss:0.580596	train-auc:0.948236	train-logloss:0.548895
[5]	valid-auc:0.87913	valid-logloss:0.561121	train-auc:0.953885	train-logloss:0.523995
[6]	valid-auc:0.876578	valid-logloss:0.547121	train-auc:0.95824	train-logloss:0.503212
[7]	valid-auc:0.884864	valid-logloss:0.533082	train-auc:0.959987	train-logloss:0.485634
[8]	valid-auc:0.885732	valid-logloss:0.519317	train-auc:0.961372	train-logloss:0.468357
[9]	valid-auc:0.890835	valid-logloss:0.508348	train-auc:0.964079	train-logloss:0.453186
[10]	valid-auc:0.898543	valid-logloss:0.495663	train-auc:0.967632	train-logloss:0.437174
[11]	valid-auc: